## Load datasets

In [1]:
import pandas as pd

# files = {
#     'none': pd.read_csv('../deepseek-coder-6.7b-instruct/deepseek-coder-6.7b-instruct.csv'),
#     'cot' : pd.read_csv('../deepseek-coder-6.7b-instruct/CoT_deepseek-coder-6.7b-instruct.csv'),
#     'cot_ont' : pd.read_csv('../deepseek-coder-6.7b-instruct/CoT_ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'cot_ont_fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_CoT_ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'cot_fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_CoT_deepseek-coder-6.7b-instruct.csv'),
#     'ont' : pd.read_csv('../deepseek-coder-6.7b-instruct/ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'ont_fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_ont_rag_deepseek-coder-6.7b-instruct.csv'),
#     'fs' : pd.read_csv('../deepseek-coder-6.7b-instruct/FS_deepseek-coder-6.7b-instruct.csv'),
# }

gold_standard = pd.read_csv('../datafiles/lcquad_test_gold.csv')
gold_standard = gold_standard.sample(n=1000, random_state=42)

In [2]:
#model_output = pd.read_csv('../datafiles/deepseek_output_21_01_2024.csv')
#model_output = pd.read_csv('../datafiles/codellama_output_22_01_2024.csv')
#model_output = pd.read_csv('../datafiles/deepseek_combined_fs8_schemalast_corrected_25_01_2024.csv')
#model_output = pd.read_csv('../datafiles/deepseek_combined_fs8_corrected_25_01_2024.csv')
#model_output =  pd.read_csv('../datafiles/deepseek_combined_corrected_23_01_2024.csv')
#model_output = pd.read_csv('../datafiles/gpt-3.5_30_01_2024.csv')
#model_output = pd.read_csv('../datafiles/deepseek_1k_02_02_2024.csv')
model_output = pd.read_csv('../datafiles/CodeLlama_1k_14_02_2024.csv')

In [3]:
model_output = model_output.fillna('none')

In [4]:
model_output['model'] = 'deepseek'

## Preprocess

In [5]:
# String to python data structure
model_output['evaluated'] = model_output['output'].apply(eval)
gold_standard['evaluated'] = gold_standard['output'].apply(eval)

In [6]:
def preprocess(output):
    _result = set()
    if 'head' in output and 'results' in output:
        subset = set()
        for element in output['results']['bindings']:
            subset |= ({element[x]['value'] for x in element.keys()})
        _result = subset
    elif 'head' in output and 'boolean' in output:
        _result = output['boolean']
    return _result

In [7]:
def check_well_constructed(output):
    if 'output error' in output:
        return False
    return True

In [8]:
def type_of_consult(consult):
    result = 'NONE'
    if 'COUNT' in consult: result = 'COUNT'
    elif 'ASK' in consult: result = 'ASK'
    elif 'ORDER BY' in consult: result = 'ORDER_BY'
    elif 'SELECT' in consult: result = 'SELECT'
    return result

In [9]:
model_output['curated'] = model_output['evaluated'].apply(preprocess)

In [10]:
gold_standard['curated'] = gold_standard['evaluated'].apply(preprocess)

In [11]:
#model_output['method'] = 'all'

### Code Execution Eval

In [12]:
grouped = model_output.groupby('method')
ex_eval = pd.DataFrame()
for i, (method, group) in enumerate(grouped):
    comparison = group['curated'].values == gold_standard['curated'].values
    print(f"Accuracy for {method}: {comparison.mean()}")
    comparison = pd.DataFrame(comparison)
    ex_eval = pd.concat([ex_eval, comparison])

Accuracy for CoT: 0.032
Accuracy for CoTont_rag: 0.045
Accuracy for FS: 0.41
Accuracy for FSCoT: 0.124
Accuracy for FSCoTont_rag: 0.129
Accuracy for FSont_rag: 0.332
Accuracy for none: 0.009
Accuracy for ont_rag: 0.013


In [13]:
model_output['evaluation'] = ex_eval[0].to_list()
model_output['evaluation'] = model_output['evaluation'].astype(int)

In [14]:
print(model_output[['method','evaluation']].groupby('method').mean().to_latex()) 

\begin{tabular}{lr}
\toprule
 & evaluation \\
method &  \\
\midrule
CoT & 0.129000 \\
CoTont_rag & 0.332000 \\
FS & 0.410000 \\
FSCoT & 0.009000 \\
FSCoTont_rag & 0.013000 \\
FSont_rag & 0.124000 \\
none & 0.032000 \\
ont_rag & 0.045000 \\
\bottomrule
\end{tabular}



In [15]:
model_output['type'] = gold_standard.sparql_query.apply(type_of_consult).to_list() * 8

In [16]:
model_output[['method','evaluation','type']].groupby(['method', 'type']).describe().round(2)

evaluation                                     
                         count  mean   std  min  25%  50%  75%  max
method       type                                                  
CoT          ASK          83.0  0.05  0.22  0.0  0.0  0.0  0.0  1.0
             COUNT       123.0  0.09  0.29  0.0  0.0  0.0  0.0  1.0
             SELECT      794.0  0.14  0.35  0.0  0.0  0.0  0.0  1.0
CoTont_rag   ASK          83.0  0.28  0.45  0.0  0.0  0.0  1.0  1.0
             COUNT       123.0  0.33  0.47  0.0  0.0  0.0  1.0  1.0
             SELECT      794.0  0.34  0.47  0.0  0.0  0.0  1.0  1.0
FS           ASK          83.0  0.48  0.50  0.0  0.0  0.0  1.0  1.0
             COUNT       123.0  0.37  0.48  0.0  0.0  0.0  1.0  1.0
             SELECT      794.0  0.41  0.49  0.0  0.0  0.0  1.0  1.0
FSCoT        ASK          83.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0
             COUNT       123.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0
             SELECT      794.0  0.01  0.11  0.0  0.0  0.0  0.0  1.0
FSCoTont_rag ASK          83.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0
             COUNT       123.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0
             SELECT      794.0  0.02  0.13  0.0  0.0  0.0  0.0  1.0
FSont_rag    ASK          83.0  0.11  0.31  0.0  0.0  0.0  0.0  1.0
             COUNT       123.0  0.12  0.33  0.0  0.0  0.0  0.0  1.0
             SELECT      794.0  0.13  0.33  0.0  0.0  0.0  0.0  1.0
none         ASK          83.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0
             COUNT       123.0  0.02  0.13  0.0  0.0  0.0  0.0  1.0
             SELECT      794.0  0.04  0.19  0.0  0.0  0.0  0.0  1.0
ont_rag      ASK          83.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0
             COUNT       123.0  0.02  0.13  0.0  0.0  0.0  0.0  1.0
             SELECT      794.0  0.05  0.23  0.0  0.0  0.0  0.0  1.0

In [17]:
model_output[['model', 'method','evaluation','type']].groupby(['model','method', 'type']).mean().round(2)

evaluation
model    method       type              
deepseek CoT          ASK           0.05
                      COUNT         0.09
                      SELECT        0.14
         CoTont_rag   ASK           0.28
                      COUNT         0.33
                      SELECT        0.34
         FS           ASK           0.48
                      COUNT         0.37
                      SELECT        0.41
         FSCoT        ASK           0.00
                      COUNT         0.00
                      SELECT        0.01
         FSCoTont_rag ASK           0.00
                      COUNT         0.00
                      SELECT        0.02
         FSont_rag    ASK           0.11
                      COUNT         0.12
                      SELECT        0.13
         none         ASK           0.00
                      COUNT         0.02
                      SELECT        0.04
         ont_rag      ASK           0.00
                      COUNT         0.02
                      SELECT        0.05

In [18]:
output = model_output[['model', 'method','evaluation','type']].groupby(['model','method', 'type']).mean() * 100

In [19]:
print(output.to_latex())

\begin{tabular}{lllr}
\toprule
 &  &  & evaluation \\
model & method & type &  \\
\midrule
\multirow[t]{24}{*}{deepseek} & \multirow[t]{3}{*}{CoT} & ASK & 4.819277 \\
 &  & COUNT & 8.943089 \\
 &  & SELECT & 14.357683 \\
\cline{2-4}
 & \multirow[t]{3}{*}{CoTont_rag} & ASK & 27.710843 \\
 &  & COUNT & 33.333333 \\
 &  & SELECT & 33.753149 \\
\cline{2-4}
 & \multirow[t]{3}{*}{FS} & ASK & 48.192771 \\
 &  & COUNT & 36.585366 \\
 &  & SELECT & 40.931990 \\
\cline{2-4}
 & \multirow[t]{3}{*}{FSCoT} & ASK & 0.000000 \\
 &  & COUNT & 0.000000 \\
 &  & SELECT & 1.133501 \\
\cline{2-4}
 & \multirow[t]{3}{*}{FSCoTont_rag} & ASK & 0.000000 \\
 &  & COUNT & 0.000000 \\
 &  & SELECT & 1.637280 \\
\cline{2-4}
 & \multirow[t]{3}{*}{FSont_rag} & ASK & 10.843373 \\
 &  & COUNT & 12.195122 \\
 &  & SELECT & 12.594458 \\
\cline{2-4}
 & \multirow[t]{3}{*}{none} & ASK & 0.000000 \\
 &  & COUNT & 1.626016 \\
 &  & SELECT & 3.778338 \\
\cline{2-4}
 & \multirow[t]{3}{*}{ont_rag} & ASK & 0.000000 \\
 &  & COUNT

In [20]:
output['evaluation', ]

KeyError: ('evaluation',)

In [21]:
model_output['valid'] = model_output['output'].apply(check_well_constructed)

In [22]:
print(model_output[['method','valid']].groupby('method').mean().to_latex())

\begin{tabular}{lr}
\toprule
 & valid \\
method &  \\
\midrule
CoT & 0.894000 \\
CoTont_rag & 0.857000 \\
FS & 0.995000 \\
FSCoT & 0.845000 \\
FSCoTont_rag & 0.847000 \\
FSont_rag & 0.958000 \\
none & 0.943000 \\
ont_rag & 0.940000 \\
\bottomrule
\end{tabular}

